In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, normalized_mutual_info_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier, GradientBoostingRegressor
import pandas as pd
import numpy as np
%matplotlib inline

In [155]:
train = pd.DataFrame.from_csv('train.tsv')
#train = train.sample(frac=0.01, random_state=3)

target = train['y']
#train = train.drop(['y'], axis=1)

test = pd.DataFrame.from_csv('test.tsv')
f60_train = train['f60']
f60_test = test['f60']

In [247]:
test['y'] = 0
test = test.set_index(np.linspace(0, test.shape[0]-1, test.shape[0]).astype(int))
test['index1'] = test.index

In [248]:
for index in test['index1']:
    week = test.loc[(test['index1']==index)]['week'].values[0]
    item = test.loc[(test['index1']==index)]['item_id'].values[0]
    A = test.loc[(test['item_id'] == item) & (test['week']-test['shift']==week)]
    if A.empty == False:        
        i = A.index[0]
        test.set_value(index, 'y', A.loc[test['index1']==i]['f60'].values[0]*1.61)


In [249]:
test[test['y']!=0].shape

(1335, 7)

In [252]:
test[:5]

,year,week,shift,item_id,f60,y,index1
0,2015,3,3,20447918,1290.0,1497,0
1,2015,3,3,20447902,21307.0,27033,1
2,2015,3,3,20447732,285904.0,297560,2
3,2015,3,3,20443951,33970.0,29471,3
4,2015,3,3,20443944,0.0,16,4


In [253]:
my_sub_24 = pd.read_csv("my_submission_24.tsv")

In [269]:
sub = test.drop(['year', 'week', 'shift', 'item_id', 'f60'], axis=1)

In [270]:
sub[-5:]

,y,index1
2011,0,2011
2012,0,2012
2013,0,2013
2014,0,2014
2015,0,2015


In [271]:
my_sub_24['index1'] = my_sub_24.index
my_sub_24[-5:]

,Num,y,index1
2011,351524,42423.030,2011
2012,351525,17370.545,2012
2013,351526,14606.590,2013
2014,351527,384.225,2014
2015,351528,4682.990,2015


In [272]:
for index in sub['index1']:
    if sub.loc[(sub['index1']==index)].values[0][0] == 0:
        sub.set_value(index, 'y', my_sub_24.loc[my_sub_24['index1']==index]['y'].values[0])        

In [273]:
sub[-5:]

,y,index1
2011,42423,2011
2012,17370,2012
2013,14606,2013
2014,384,2014
2015,4682,2015


In [274]:
sample_submission = pd.read_csv("sample_submission.tsv")
preds = sub['y']
sample_submission['y'] = preds
print sample_submission[sample_submission['y'] < 0]
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)
sample_submission.to_csv("my_submission_24_leak.tsv", sep=',', index=False)

Empty DataFrame
Columns: [Num, y]
Index: []
